# QM Dataset

This is a base class for 'quantum mechanical' datasets. It generates graph properties from a xyz-file, which
stores atomic coordinates.

Additionally, it should be possible to generate approximate chemical bonding information via `OpenBabel`, if this
additional package is installed.
The class inherits from `MemoryGraphDataset`.

At the moment, there is no connection to `MoleculeNetDataset` since usually for geometric data, the usage is
related to learning quantum properties like energy, orbitals or forces and no "chemical" feature information is
required.

For demonstration, we make an artifical table of coordinates and some target values and store them to file.

In [1]:
import os
os.makedirs("ExampleQM", exist_ok=True)
xyz_list = [
    "3\n\nC -0.8513 1.7563 0.5028\nC -1.1415 0.2664 0.4371\nC -0.7681 -0.3186 -0.9144\n",
    "4\n\nC 2.4098 0.5514 -2.1836\nC 2.5000 -0.4800 -1.0676\nC 1.1575 -0.7559 -0.3909\nN 0.6356 0.4257 0.2851\n",
    "1\n\nC 0.0 0.0 0.0\n"
]
xyz_data = "".join(xyz_list)
with open("ExampleQM/qm.xyz", "w") as f:
    f.write(xyz_data)

Or if single files are used

In [2]:
os.makedirs("ExampleQM/XYZ_files", exist_ok=True)
for i, x in enumerate(xyz_list):
    with open("ExampleQM/XYZ_files/mol_%i.xyz" % i, "w") as f:
        f.write(x)
csv_info = "ID,files,energy\n0,mol_0.xyz,-13.0\n1,mol_1.xyz,-20.0\n2,mol_2.xyz,-34.0"
with open("ExampleQM/qm.csv", "w") as f:
    f.write(csv_info)

The file structure is:


```bash
├── ExampleQM
    ├── qm.csv
    ├── XYZ_files  # Need a qm.csv if singe files are used.
    │   ├── *.*
    │   └── ... 
    ├── qm.xyz
    └── qm.sdf  # After prepare_data
```

##  1. Initialization

To load the dataset from memeory the `QMDataset` class requires the information of the directory the data is in and the name of the xyz-file. Also required is to provide a name of the dataset. If no single xyz file is available, a csv file of the same base-name must have also a column of files-names in addition to labels.

In [3]:
import numpy as np
from kgcnn.data.qm import QMDataset
dts = QMDataset(
    file_name="qm.xyz",
    file_directory="XYZ_files",
    data_directory="ExampleQM", 
    dataset_name="ExampleQM"
)

## 2. Prepare Data

Prepare Data generates a single XYZ file if it is not available and also tries to maks a mol SDF file from XYZ information via `OpenBabel` if installed.

In [4]:
dts.prepare_data(
    overwrite=True,
    xyz_column_name="files",  # Delete qm.xyz file to see if it is generated from single files
    make_sdf=True
)

INFO:kgcnn.data.ExampleQM:Reading single xyz-file.
INFO:kgcnn.data.ExampleQM:Converting xyz to mol information.


## 3. Read Data

The information from XYZ file and optinally also form SDF and CSV files are read from disk.

In [5]:
dts.read_in_memory(
    label_column_name="energy"
)

INFO:kgcnn.data.ExampleQM:Parsing mol information ...


TypeError: __init__() got an unexpected keyword argument 'add_hydrogen'

## 4. Check dataset and graphs

With openbabel QMDataset has both geometric and apporximate bond information.

In [ ]:
dts.obtain_property("node_coordinates")

The graph then must be constructed with edges based on pairwise distances:

In [ ]:
dts.map_list("set_range", 
    max_distance=2,
    max_neighbours=15,
    do_invert_distance=False, 
    self_loops=False, 
    exclusive=True
)

In [ ]:
dts.obtain_property("range_attributes")

In [ ]:
dts.obtain_property("range_indices")

If openbabel is available, the also bond information are given.

In [ ]:
dts.obtain_property("edge_indices")

In [ ]:
dts.obtain_property("edge_number")

In [ ]:
# We can also clean the dataset to ensure there are no un-connected graphs or empty arrays.
dts.clean(inputs=["edge_number", "edge_indices"])

Cleaned dataset without empty or zero-length nodes/edges

In [ ]:
dts.obtain_property("edge_indices")

In [ ]:
dts[0]

## 5. Extensive scaler

For extensive targets like energies, it can be helpful to remove an energy offset which is like a sum of atomization energies. 

In [ ]:
from kgcnn.scaler.mol import ExtensiveMolecularScaler

In [ ]:
scaler = ExtensiveMolecularScaler()
energies_scaled = scaler.fit_transform(dts.obtain_property("node_number"), dts.obtain_property("graph_labels"))
energies_scaled

In [ ]:
scaler._plot_predict(dts.obtain_property("node_number"), dts.obtain_property("graph_labels"))